In [1]:
import os
import torch.autograd
import torch.nn as nn



In [2]:
from Model.CVAE_PG_GAN import Gen_Block
# z=torch.randn(64,100)
# y=torch.randn(64,10)
# z0=torch.concat([z,y],dim=1)

# linear=nn.Linear(110,3*16*16)
gen_block=Gen_Block(3,32,3,1,1,100,10)
print(gen_block)
# x=linear(z0)
# print(x.shape)
# x=x.view(x.size(0),-1,16,16)
# print(x.shape)
# x1=gen_block(x,z0)
# print(x1.shape)


Gen_Block(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (cbn1): ConditionalBatchNorm2d_for_skip_and_shared(
    (bn): BatchNorm2d(32, eps=0.0001, momentum=0.1, affine=False, track_running_stats=True)
    (gain): Linear(in_features=110, out_features=32, bias=False)
    (bias): Linear(in_features=110, out_features=32, bias=False)
  )
  (cbn2): ConditionalBatchNorm2d_for_skip_and_shared(
    (bn): BatchNorm2d(32, eps=0.0001, momentum=0.1, affine=False, track_running_stats=True)
    (gain): Linear(in_features=110, out_features=32, bias=False)
    (bias): Linear(in_features=110, out_features=32, bias=False)
  )
  (self_attn): Self_Attn(
    (conv1x1_theta): Conv2d(32, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv1x1_phi): Conv2d(32, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
    

In [3]:
import torch
from Model.CVAE_PG_GAN import CVAE
y=torch.randn(128,3,64,64)
cvae = CVAE(nz=100,
			 imSize=64,
             block_num=4,
             in_channel=3,
			 fSize=32,
             numLabels=10,
			 device='cpu')
print(cvae)
mu, log_var, y =cvae.encode(y)
z = cvae.re_param(mu, log_var)
reconstruction = cvae.decode(y,z,1,0.5)
print(reconstruction.shape)


CVAE(
  (encLogVar): Linear(in_features=4096, out_features=100, bias=True)
  (encMu): Linear(in_features=4096, out_features=100, bias=True)
  (encY): Linear(in_features=4096, out_features=10, bias=True)
  (enc_layers): ModuleList(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (2): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (3): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  )
  (enc_attn_block): Self_Attn(
    (conv1x1_theta): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv1x1_phi): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv1x1_g): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (conv1x1_attn): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (softma

e:\Project\ZLTProgram\GTSRB\Model\CVAE_PG_GAN.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y = F.softmax(self.encY(x.detach()))


In [4]:
from Model.CVAE_PG_GAN import DISCRIMINATOR
discriminator= DISCRIMINATOR(64,4,32,1,'cpu')
print(discriminator)
dis=discriminator(reconstruction,1,0.5)
print(dis)


DISCRIMINATOR(
  (dis_blocks): ModuleList(
    (0): Dis_Block(
      (layer1): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): LeakyReLU(negative_slope=0.2)
      )
      (layer2): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): LeakyReLU(negative_slope=0.2)
      )
    )
    (1): Dis_Block(
      (layer1): Sequential(
        (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): LeakyReLU(negative_slope=0.2)
      )
      (layer2): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): LeakyReLU(negative_slope=0.2)
      )
    )
    (2): Dis_Block(
      (layer1): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): LeakyReLU(negative_slope=0.2)
      )
      (layer2): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), p